In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Loading data
df = pd.read_csv("../data/clean/clean_dataset.csv",sep=";",encoding="utf-8")

## Dataframe "genders"

In [4]:
gender_values = df["gender"].unique()
gender_values

<StringArray>
['Male', 'Female']
Length: 2, dtype: str

In [5]:
genders = pd.DataFrame({
    "gender_id": range(1, len(gender_values) + 1),
    "gender": gender_values
})
genders

,gender_id,gender
0,1,Male
1,2,Female


In [6]:
#Exporting
genders.to_csv(
    "../data/clean/genders.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "travel_types"

In [7]:
type_of_travel_values = df["type_of_travel"].unique()
type_of_travel_values

<StringArray>
['Personal Travel', 'Business Travel']
Length: 2, dtype: str

In [8]:
travel_types = pd.DataFrame({
    "travel_type_id": range(1, len(type_of_travel_values) + 1),
    "type_of_travel": type_of_travel_values
})
travel_types

,travel_type_id,type_of_travel
0,1,Personal Travel
1,2,Business Travel


In [9]:
#Exporting
travel_types.to_csv(
    "../data/clean/travel_types.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "customer_types"

In [10]:
customer_type_values = df["customer_type"].unique()
customer_type_values

<StringArray>
['Loyal Customer', 'Disloyal Customer']
Length: 2, dtype: str

In [11]:
customer_types = pd.DataFrame({
    "customer_type_id": range(1, len(customer_type_values) + 1),
    "customer_type": customer_type_values
})
customer_types

,customer_type_id,customer_type
0,1,Loyal Customer
1,2,Disloyal Customer


In [12]:
#Exporting
customer_types.to_csv(
    "../data/clean/customer_types.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "satisfactions"

In [13]:
satisfaction_values = df["satisfaction"].unique()
satisfaction_values

<StringArray>
['Neutral Or Dissatisfied', 'Satisfied']
Length: 2, dtype: str

In [14]:
satisfactions = pd.DataFrame({
    "satisfaction_id": range(1, len(satisfaction_values) + 1),
    "satisfaction": satisfaction_values
})
satisfactions

,satisfaction_id,satisfaction
0,1,Neutral Or Dissatisfied
1,2,Satisfied


In [15]:
#Exporting
satisfactions.to_csv(
    "../data/clean/satisfactions.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "classes"

In [16]:
class_values = df["class"].unique()
class_values

<StringArray>
['Eco Plus', 'Business', 'Eco']
Length: 3, dtype: str

In [17]:
classes = pd.DataFrame({
    "class_id": range(1, len(class_values) + 1),
    "class": class_values
})
classes

,class_id,class
0,1,Eco Plus
1,2,Business
2,3,Eco


In [18]:
#Exporting
classes.to_csv(
    "../data/clean/classes.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "flights"

In [19]:
flight_values = df["flight_distance"].unique()
flight_values

array([ 460,  235, 1142, ...,  974, 1479,  400], shape=(3802,))

In [20]:
flights = pd.DataFrame({
    "flight_id": range(1, len(flight_values) + 1),
    "flight_distance": flight_values
})
flights

,flight_id,flight_distance
0,1,460
1,2,235
2,3,1142
3,4,562
4,5,214
...,...,...
3797,3798,1309
3798,3799,1219
3799,3800,974
3800,3801,1479


In [21]:
#Exporting
flights.to_csv(
    "../data/clean/flights.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "flight_classes"

In [22]:
#creating bridge table
flights_classes = (
    flights
    .assign(key=1)
    .merge(classes.assign(key=1), on="key")
    .drop("key", axis=1)
)

flights_classes.head()

,flight_id,flight_distance,class_id,class
0,1,460,1,Eco Plus
1,1,460,2,Business
2,1,460,3,Eco
3,2,235,1,Eco Plus
4,2,235,2,Business


In [23]:
flights_classes = flights_classes[["flight_id", "class_id"]].copy()

flights_classes.insert(
    0,
    "flight_class_id",
    range(1, len(flights_classes) + 1)
)

flights_classes.head()

,flight_class_id,flight_id,class_id
0,1,1,1
1,2,1,2
2,3,1,3
3,4,2,1
4,5,2,2


In [24]:
#Exporting
flights_classes.to_csv(
    "../data/clean/flights_classes.csv",
    index=False,sep=";",encoding="utf-8"
)

## Dataframe "passenger"

In [25]:
gender_map = dict(zip(genders["gender"], genders["gender_id"]))
travel_type_map = dict(zip(travel_types["type_of_travel"], travel_types["travel_type_id"]))
customer_type_map = dict(zip(customer_types["customer_type"], customer_types["customer_type_id"]))
satisfaction_map = dict(zip(satisfactions["satisfaction"], satisfactions["satisfaction_id"]))
flight_map = dict(zip(flights["flight_distance"], flights["flight_id"]))

In [26]:
passengers = pd.DataFrame({
    "passenger_id": range(1, len(df) + 1),  # PK (one passenger profile per row)
    "age": df["age"].astype("int64", errors="ignore"),

    "flight_id": df["flight_distance"].map(flight_map),
    "gender_id": df["gender"].map(gender_map),
    "travel_type_id": df["type_of_travel"].map(travel_type_map),
    "satisfaction_id": df["satisfaction"].map(satisfaction_map),
    "customer_type_id": df["customer_type"].map(customer_type_map),
})

passengers.head()

,passenger_id,age,flight_id,gender_id,travel_type_id,satisfaction_id,customer_type_id
0,1,13,1,1,1,1,1
1,2,25,2,1,2,1,2
2,3,26,3,2,2,2,1
3,4,25,4,2,2,1,1
4,5,61,5,1,2,2,1


In [27]:
# FK integrity checks
# 1) Missing values in FKs
print(passengers[["flight_id","gender_id","travel_type_id","satisfaction_id","customer_type_id"]].isna().sum())

# 2) Ensure passenger_id is unique
print("passenger_id unique:", passengers["passenger_id"].is_unique)

# 3) (Optional) check all FK ids exist in parent tables
assert set(passengers["gender_id"].dropna()).issubset(set(genders["gender_id"]))
assert set(passengers["travel_type_id"].dropna()).issubset(set(travel_types["travel_type_id"]))
assert set(passengers["customer_type_id"].dropna()).issubset(set(customer_types["customer_type_id"]))
assert set(passengers["satisfaction_id"].dropna()).issubset(set(satisfactions["satisfaction_id"]))
assert set(passengers["flight_id"].dropna()).issubset(set(flights["flight_id"]))

flight_id           0
gender_id           0
travel_type_id      0
satisfaction_id     0
customer_type_id    0
dtype: int64
passenger_id unique: True


In [32]:
passengers["passenger_id"].unique

<bound method Series.unique of 0              1
1              2
2              3
3              4
4              5
           ...  
103899    103900
103900    103901
103901    103902
103902    103903
103903    103904
Name: passenger_id, Length: 103904, dtype: int64>

In [28]:
#Exporting
passengers.to_csv(
    "../data/clean/passengers.csv",
    index=False,
    sep=";",
    encoding="utf-8"
)

## Dataframe "surveys"

In [29]:
surveys = pd.DataFrame({
    # Keep original dataset id as the survey primary key
    "id": df["id"].astype(int),

    # FK: must exist in passengers.passenger_id
    "passenger_id": range(1, len(df) + 1),

    "inflight_wifi_service": df["inflight_wifi_service"],
    "departure_arrival_time_convenient": df["departure_arrival_time_convenient"],
    "ease_of_online_booking": df["ease_of_online_booking"],
    "gate_location": df["gate_location"],
    "food_and_drink": df["food_and_drink"],
    "online_boarding": df["online_boarding"],
    "seat_comfort": df["seat_comfort"],
    "inflight_entertainment": df["inflight_entertainment"],
    "on_board_service": df["on-board_service"],
    "leg_room_service": df["leg_room_service"],
    "baggage_handling": df["baggage_handling"],
    "checkin_service": df["checkin_service"],
    "inflight_service": df["inflight_service"],
    "cleanliness": df["cleanliness"],
    "departure_delay_in_minutes": df["departure_delay_in_minutes"],
    "arrival_delay_in_minutes": df["arrival_delay_in_minutes"],
})

surveys.head()


,id,passenger_id,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,on_board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes
0,70172,1,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18
1,5047,2,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6
2,110028,3,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0
3,24026,4,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9
4,119299,5,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0


In [30]:
# Foreign key integrity checks
# 1) passenger_id in surveys must exist in passengers
missing_fk = set(surveys["passenger_id"]) - set(passengers["passenger_id"])
print("Missing passenger_id FKs:", len(missing_fk))

# 2) survey PK should be unique
print("Survey id unique:", surveys["id"].is_unique)
print(surveys.isna().sum())


Missing passenger_id FKs: 0
Survey id unique: True
id                                   0
passenger_id                         0
inflight_wifi_service                0
departure_arrival_time_convenient    0
ease_of_online_booking               0
gate_location                        0
food_and_drink                       0
online_boarding                      0
seat_comfort                         0
inflight_entertainment               0
on_board_service                     0
leg_room_service                     0
baggage_handling                     0
checkin_service                      0
inflight_service                     0
cleanliness                          0
departure_delay_in_minutes           0
arrival_delay_in_minutes             0
dtype: int64


In [31]:
#Exporting
surveys.to_csv(
    "../data/clean/surveys.csv",
    index=False,
    sep=";",
    encoding="utf-8"
)